In [ ]:
using Plots
import Plots: px
theme(:default)
ENV["LINES"] = 20

In [ ]:
include("parser.jl")
include("models.jl")
gpu()
download(SIGDataSet)

In [ ]:
data  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-train-high"));
vocab = Vocabulary(data)
edata = encode(data,vocab);
test  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-test"));
tdata = encode(test,vocab);
dictionary = [parseDataLine(line) for line in  eachline("./data/unimorph/spa/spa") if line != ""];

In [ ]:
vocab.chars.toElement;

In [ ]:
vocab.specialIndices

In [ ]:
trainsfs = unique(map(x->join(x.surface),data));
testsfs  =  unique(map(x->join(x.surface),test)); 
dictsfs  = unique([map(x->join(x.surface),dictionary); trainsfs; testsfs]);

In [ ]:
morph = EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
train_ae!(morph, data, vocab; optim=Adam(), epoch=50)

In [ ]:
morphv =  EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
copytoparams(morphv.encoder, morph.encoder)
copytoparams(morphv.Wμ, morph.Wμ)
copytoparams(morphv.Wσa, morph.Wσa)
copytoparams(morphv.Wμa, morph.Wμa)
copytoparams(morphv.Wσ, morph.Wσ)
copytoparams(morphv.Weaμ, morph.Weaμ)
copytoparams(morphv.Weaσ, morph.Weaσ)
morphv.decoder.embedding = morphv.encoder.embedding
train_vae!(morphv, data, vocab; optim=Adam(), epoch=45, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=3)

In [ ]:
samples = sample(morphv, vocab, edata; N=10000);

In [ ]:
unique(samples[findall([s ∈ trainsfs for s in samples])])

In [ ]:
length(samples[findall([s ∈ trainsfs for s in samples])])

In [ ]:
unique(samples[findall([s ∈ testsfs for s in samples])])

In [ ]:
length(samples[findall([s ∈ testsfs for s in samples])])

In [ ]:
unique(length(samples[findall([s ∈ dictsfs for s in samples])]))

In [ ]:
sampleinter(morphv, vocab, data; N=10)

In [ ]:
au, sigma, mu = calc_au(morphv,tdata; delta=0.01)

In [ ]:
mi = calc_mi(morphv,tdata)

In [ ]:
calc_ppl(morphv,tdata)

In [ ]:
atts = attentions(morph,edata, vocab);

In [ ]:
function getsingle(sfs,exs,perms, αs,i) 
    join(vocab.chars[sfs[i]]), map(e->join(vocab.chars[e]),exs[i][perms[i]]), map(x->x[:,perms[i],i],αs)
end

In [ ]:
i = rand(1:length(atts))
s1,e1,α1 =  getsingle(atts[i]...,rand(1:16))
ys = 1:512
h1 = heatmap(e1, ys, α1.αu; size=(1200,800), xtickfont = font(10, "Halvetica"), title= s1 * ", mu", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);
h2 = heatmap(e1, ys, α1.ασ ;size=(1200,800), xtickfont = font(10, "Halvetica"), title=s1 * ", sigma", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);

In [ ]:
println(s1);println(e1)

In [ ]:
h1

# To-Dos
1. Add unrelated surface forms to the examplars too see whether the attention mappings are meaningful
2. Comparing with Normal Vae
3. Different Sampling Function on LM ($top_k$, temperature)
4. Attention in Decoder
5. Attention to the hidden state sequence, not just to the final state.
6. Metric for testing generation quality?

In [56]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [57]:
morph3=nothing; morphv=nothing; morph=nothing; morph2=nothing; KnetLayers.gc();

In [58]:
morph2 = VAE(length(vocab.chars); H=512, E=16, Z=16)
train_ae!(morph2, data, vocab; optim=Adam(), B=16, epoch=40);

(loss = 25.483113110351564,)
(loss = 19.472908088684083,)
(loss = 17.730347187805176,)
(loss = 16.60444072570801,)
(loss = 15.403253541564942,)
(loss = 14.089014128112794,)
(loss = 13.170593530273438,)
(loss = 12.13779828338623,)
(loss = 10.977761215209961,)
(loss = 9.8521598487854,)
(loss = 8.679309171295166,)
(loss = 7.643301724243164,)
(loss = 7.062928867340088,)
(loss = 6.286370854187012,)
(loss = 5.740640664291382,)
(loss = 5.037542947387696,)
(loss = 4.522800597000122,)
(loss = 3.9606379707336425,)
(loss = 3.556455333328247,)
(loss = 3.234172971343994,)
(loss = 2.839251784324646,)
(loss = 2.6254156524658203,)
(loss = 2.4203680517196657,)
(loss = 2.1923937099456787,)
(loss = 2.0407046070098875,)
(loss = 1.8649052600860596,)
(loss = 1.7781351954460145,)
(loss = 1.6920295616149903,)
(loss = 1.55064642868042,)
(loss = 1.4903384167194367,)
(loss = 1.3627260312557221,)
(loss = 1.2180920554161072,)
(loss = 1.2440031058311463,)
(loss = 1.2316336155414582,)
(loss = 1.1179177912712097,)
(l

In [59]:
morph3 = VAE(length(vocab.chars); H=512, E=16, Z=16)
copytoparams(morph3.encoder, morph2.encoder)
copytoparams(morph3.Wμ, morph2.Wμ)
copytoparams(morph3.Wσ, morph2.Wσ)
morph3.decoder.embedding = morph3.encoder.embedding
train_vae!(morph3, data, vocab; B=16, optim=Adam(), epoch=45, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=3)

(kl_weight = 0.1f0, fbr = 3, loss = 15.443869478607178)
(kl_weight = 0.2f0, fbr = 3, loss = 14.923171812438964)
(kl_weight = 0.3f0, fbr = 3, loss = 15.238884265136718)
(kl_weight = 0.4f0, fbr = 3, loss = 15.980613064575195)
(kl_weight = 0.5f0, fbr = 3, loss = 16.71773899230957)
(kl_weight = 0.6f0, fbr = 3, loss = 17.226584687805175)
(kl_weight = 0.70000005f0, fbr = 3, loss = 17.68540315093994)
(kl_weight = 0.8000001f0, fbr = 3, loss = 18.103392684936523)
(kl_weight = 0.9000001f0, fbr = 3, loss = 18.565587379455568)
(kl_weight = 1.0f0, fbr = 3, loss = 18.74658327026367)
(kl_weight = 1.0f0, fbr = 3, loss = 18.92175535583496)
(kl_weight = 1.0f0, fbr = 3, loss = 18.48210305633545)
(kl_weight = 1.0f0, fbr = 3, loss = 18.057821995544433)
(kl_weight = 1.0f0, fbr = 3, loss = 17.72012526397705)
(kl_weight = 1.0f0, fbr = 3, loss = 17.328961029052735)
(kl_weight = 1.0f0, fbr = 3, loss = 17.057035917663573)
(kl_weight = 1.0f0, fbr = 3, loss = 16.77470923461914)
(kl_weight = 1.0f0, fbr = 3, loss = 

In [60]:
samples = sample(morph3, vocab, edata; N=10000, useprior=true)

10005-element Array{String,1}:
 "incoludiéis"  
 "no repante"   
 "matizáramos"  
 "rebanecidos"  
 "enfastaremos" 
 "colraban"     
 "ocapase"      
 "conoqueabais" 
 ⋮              
 "foctamos"     
 "aludió"       
 "catamos"      
 "cirre"        
 "lacrimare"    
 "diluiremos"   
 "cronometraron"

In [61]:
samples[findall([s ∈ trainsfs for s in samples])]

2996-element Array{String,1}:
 "perseguiríais" 
 "sobrevire"     
 "concursados"   
 "destensaran"   
 "pretiriera"    
 "traspapele"    
 "convocáramos"  
 "confeccionen"  
 ⋮               
 "desnivelara"   
 "garantizáramos"
 "mire"          
 "aludió"        
 "catamos"       
 "diluiremos"    
 "cronometraron" 

In [62]:
samples[findall([s ∈ testsfs for s in samples])]

15-element Array{String,1}:
 "enrole"       
 "convergía"    
 "desenfunda"   
 "okupara"      
 "deshidratemos"
 "asustemos"    
 "resupiese"    
 "custodiáis"   
 "atascamos"    
 "dormitaría"   
 "ofrendare"    
 "costaseis"    
 "pelaría"      
 "acuñara"      
 "aplomamos"    

In [63]:
samples[findall([s ∈ dictsfs for s in samples])]

4975-element Array{String,1}:
 "matizáramos"   
 "perseguiríais" 
 "redibujado"    
 "no arrastremos"
 "sobrevire"     
 "concursados"   
 "destensaran"   
 "entreve"       
 ⋮               
 "garantizáramos"
 "mire"          
 "aludió"        
 "catamos"       
 "lacrimare"     
 "diluiremos"    
 "cronometraron" 

In [64]:
inters = sampleinter(morph3, vocab, data; N=10)

12-element Array{String,1}:
 "conservareis"  
 "conservaría"   
 "conservaría"   
 "convergiésemos"
 "convergiésemos"
 "enternecen"    
 "empeñases"     
 "empeñases"     
 "emparedamos"   
 "empatemos"     
 "empatemos"     
 "empatemos"     

In [65]:
findall([s ∈ dictsfs for s in inters])

12-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [66]:
au, sigma, mu = calc_au(morph3, tdata; delta=0.01)

(16, Float32[0.3343065; 0.38925314; … ; 0.34903616; 0.33873308], K32(16,1)[-0.00467507⋯])

In [67]:
mi =  calc_mi(morph3,tdata)

4.1258481597900385

In [68]:
calc_ppl(morph3,tdata; nsample=500, B=16)

5.5472722464322715

In [21]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [ ]:
model = LSTM_LM(length(vocab.chars); H=768, E=16)

In [ ]:
train_rnnlm!(model, data, vocab; epoch=40, optim=Adam(), B=16)

In [ ]:
 calc_ppllm(model, tdata)

In [ ]:
samples = sample(model, vocab; N=10000)

In [ ]:
samples[findall([s ∈ trainsfs for s in samples])]

In [ ]:
samples[findall([s ∈ testsfs for s in samples])]

In [ ]:
samples[findall([s ∈ dictsfs for s in samples])]

In [ ]:
 calc_ppllm(model, edata)